# Discovery Fuentes datos.gob.es

In [1]:

import pandas as pd
import requests

### Descarga del dataset

Se utilizará un dataset público proporcionado por el Instituto Nacional de Estadística de España disponible en datos.gob.es.

In [2]:
# URL del archivo CSV
URL = "https://www.ine.es/jaxiT3/files/t/csv_bdsc/48423.csv"

# Ruta específica donde se desea guardar el archivo CSV descargado
PATH = "./tourism_spain_dataset.csv"

In [3]:
# Realizamos la solicitud HTTP para descargar el archivo
r = requests.get(URL)

# Verificamos que la solicitud fue exitosa (código de estado 200)
if r.status_code == 200:
    # Guardamos la información descargada en el archivo local
    with open(PATH, 'wb') as f:
        f.write(r.content)
    print(f"Archivo descargado exitosamente en: {PATH}")
else:
    print(f"Error al descargar el archivo. Código de estado: {r.status_code}")

Archivo descargado exitosamente en: ./tourism_spain_dataset.csv


### Carga del dataset

Cargamos nuestro dataset dentro de un dataframe de `pandas` que nos permite manipular, limpiar y visualizar fácilmente nuestros datos.

In [4]:
df = pd.read_csv(PATH, delimiter=';')

Utilizamos el método `data.info` para conocer el tipo de dato de cada columna y la cantidad de valores no nulos.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12420 entries, 0 to 12419
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Concepto turístico                12420 non-null  object
 1   Total Nacional                    12420 non-null  object
 2   Comunidades y Ciudades Autónomas  11799 non-null  object
 3   Residencia del viajero            12420 non-null  object
 4   Periodo                           12420 non-null  object
 5   Total                             11907 non-null  object
dtypes: object(6)
memory usage: 582.3+ KB


In [6]:
df.sample(10)

,Concepto turístico,Total Nacional,Comunidades y Ciudades Autónomas,Residencia del viajero,Periodo,Total
416,Número de alojamientos turísticos ocupados,Total Nacional,02 Aragón,Total,2023M07,NaN
7800,Número de noches ocupadas,Total Nacional,"17 Rioja, La",Residentes en el Extranjero,2023M06,3.000
736,Número de alojamientos turísticos ocupados,Total Nacional,"03 Asturias, Principado de",Residentes en España,2019M11,4.000
9986,Estancia media,Total Nacional,08 Castilla - La Mancha,Total,2019M07,"2,5"
6038,Número de noches ocupadas,Total Nacional,09 Cataluña,Total,2020M10,131.000
5597,Número de noches ocupadas,Total Nacional,07 Castilla y León,Total,2023M01,27.000
3439,Número de alojamientos turísticos ocupados,Total Nacional,16 País Vasco,Residentes en España,2018M11,6.000
5360,Número de noches ocupadas,Total Nacional,05 Canarias,Residentes en el Extranjero,2019M10,335.000
8564,Estancia media,Total Nacional,01 Andalucía,Residentes en España,2023M01,"2,9"
6536,Número de noches ocupadas,Total Nacional,11 Extremadura,Residentes en España,2019M07,10.000


Al ver la información del dataset nos damos cuenta que la columna `Total` debería ser un valor de tipo numérico en vez de tipo objeto. Esto sucede porque `pandas` interpreta los valores como string, ya que hay valores faltantes no codificados como `NaN` y, además, se está usando el punto como separador de miles y la coma como separador decimal, por lo que debemos reemplazar dichos caracteres y convertir los datos al tipo correspondiente.

Adicionalmente, reemplazaremos los valores nulos de la columna `Comunidades y Ciudades Autónomas` por la cadena de caracteres `"Total Nacional"` ya que se refiere a los datos obtenidos en el conjunto de todas las Comunidades y Ciudades Autónomas.

La columna `Total Nacional` sólo tiene un valor único que no aporta utilidad al estudio.

Los valores nulos de la columna `Total` se pueden rellenar con el promedio del respectivo mes.

In [7]:
df.describe()

,Concepto turístico,Total Nacional,Comunidades y Ciudades Autónomas,Residencia del viajero,Periodo,Total
count,12420,12420,11799,12420,12420,11907
unique,3,1,19,3,69,824
top,Número de alojamientos turísticos ocupados,Total Nacional,01 Andalucía,Total,2023M09,..
freq,4140,12420,621,4140,180,2189


La columna `Total Nacional` sólo tiene un valor único que no aporta utilidad al estudio.

In [8]:
df['Comunidades y Ciudades Autónomas'].unique()

array([nan, '01 Andalucía', '02 Aragón', '03 Asturias, Principado de',
       '04 Balears, Illes', '05 Canarias', '06 Cantabria',
       '07 Castilla y León', '08 Castilla - La Mancha', '09 Cataluña',
       '10 Comunitat Valenciana', '11 Extremadura', '12 Galicia',
       '13 Madrid, Comunidad de', '14 Murcia, Región de',
       '15 Navarra, Comunidad Foral de', '16 País Vasco', '17 Rioja, La',
       '18 Ceuta', '19 Melilla'], dtype=object)

In [9]:
df[['Comunidades y Ciudades Autónomas', 'Periodo', 'Concepto turístico', 'Total']].drop_duplicates().sort_values(by='Comunidades y Ciudades Autónomas')

,Comunidades y Ciudades Autónomas,Periodo,Concepto turístico,Total
207,01 Andalucía,2023M09,Número de alojamientos turísticos ocupados,NaN
8548,01 Andalucía,2018M08,Estancia media,"4,6"
8547,01 Andalucía,2018M09,Estancia media,"3,9"
8546,01 Andalucía,2018M10,Estancia media,"3,6"
8545,01 Andalucía,2018M11,Estancia media,"3,4"
...,...,...,...,...
8482,NaN,2018M05,Estancia media,"4,4"
8483,NaN,2018M04,Estancia media,"4,5"
8484,NaN,2018M03,Estancia media,"4,9"
8485,NaN,2018M02,Estancia media,"5,3"


La tabla se puede reordenar dividiendo `Periodo` en mes y año, y dividiendo `Concepto turístico` en sus 3 valores únicos con su respectivo valor de la columna `Total`, por lo tanto, se eliminaría ésta última. De esta manera, se consigue mayor legibilidad y manejo de la tabla.

In [10]:
df.sample(50).to_csv('./tourism_spain_sample.csv')

In [11]:
df.query('`Comunidades y Ciudades Autónomas`== "01 Andalucía"').sample(50).to_csv('./Andalucía_sample.csv')